In [309]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as pltly

In [310]:
# Importing data from the source
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_prognostic = fetch_ucirepo(id=16) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_prognostic.data.features 
y = breast_cancer_wisconsin_prognostic.data.targets 
  
# metadata 
source_metadata = breast_cancer_wisconsin_prognostic.metadata
print(source_metadata) 
  
# variable information 
list_variables = breast_cancer_wisconsin_prognostic.variables
print(list_variables) 


{'uci_id': 16, 'name': 'Breast Cancer Wisconsin (Prognostic)', 'repository_url': 'https://archive.ics.uci.edu/dataset/16/breast+cancer+wisconsin+prognostic', 'data_url': 'https://archive.ics.uci.edu/static/public/16/data.csv', 'abstract': 'Prognostic Wisconsin Breast Cancer Database', 'area': 'Health and Medicine', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 198, 'num_features': 33, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Outcome'], 'index_col': ['ID'], 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1995, 'last_updated': 'Sun Jan 14 2024', 'dataset_doi': '10.24432/C5GK50', 'creators': ['William Wolberg', 'W. Street', 'Olvi Mangasarian'], 'intro_paper': None, 'additional_info': {'summary': 'Each record represents follow-up data for one breast cancer case.  These are consecutive patients seen by Dr. Wolberg since 1984, and include only those cases exhibiting invasive brea

In [311]:
print("X.shape", X.shape)

# Handling Null/NaN missing values
null_value_variables = np.array([]) # initialize an empty array to store the variables with null value
for col in X:
    if(X[col].isnull().sum() != 0):
        print(col,"has null value count ", X[col].isnull().sum())
        X = X.fillna(int(X[col].mean())) # fill missing value by taking int(mean) of that column
        
        null_value_variables = np.append(null_value_variables, col)   

print("list of null value columns",null_value_variables)

# After null / nan value treatment
for null_value_variable in null_value_variables:
    print(null_value_variable,"has null value count ",X[null_value_variable].isnull().sum(), "after treating missing values as mean of the data")

X.shape (198, 33)
lymph_node_status has null value count  4
list of null value columns ['lymph_node_status']
lymph_node_status has null value count  0 after treating missing values as mean of the data


In [312]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [313]:

# transforming categorical data N and R to integers 0s and 1s
label_encoder = LabelEncoder()
# Fit label encoder and transform target variable
#print(y)
y_dash = np.ravel(y)
#print(y_dash)
#Label Encoding the features (N as 0,R as 1)
y_encoded = label_encoder.fit_transform(y_dash)
#print(y_encoded)
# splitting the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.20, random_state=42)

#Feature scaling
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [314]:
models = [LogisticRegression(), SVC(), DecisionTreeClassifier(), RandomForestClassifier(), KNeighborsClassifier()]
model_names =['Logistic_Regression', 'SVM', 'Decision_Tree_Classifier', 'Random_Forest_Classifier', 'KNN']



In [315]:
# Hyperparameters tunning
# Understanding all of the hyperparameters for parameter selection for GridSearchCV()

# listing all the parameters for our selected models
for model, model_name in zip(models, model_names):
    print("\n", model, model.get_params())

#Setting up dictionaries for the parameters for models 
param_grid_lrc = {'fit_intercept': [True, False]}
param_grid_svc = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}
param_grid_dtc = {'max_depth': list(range(1, 30))}  
param_grid_rfc = {}  
param_grid_knc = {'n_neighbors': list(range(1,15))}  

# list of dictionaries
param_grid_list = [param_grid_lrc, param_grid_svc, param_grid_dtc, param_grid_rfc, param_grid_knc]
# print(param_grid_list)


 LogisticRegression() {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}

 SVC() {'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}

 DecisionTreeClassifier() {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}

 RandomForestClassifier() {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini'

In [316]:
# Generating and saving models
import joblib

for model, model_name, param_grid in zip(models, model_names, param_grid_list):
    # fit the model using GridSearchCV Grid Search Cross Validation
    grid_search_model = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='f1')

    # training the model on the dataset
    grid_search_model.fit(X_train, y_train)

    # saving the model using joblib
    model_file_name = model_name+".joblib"
    print("Saving Model: ", model_file_name)
    joblib.dump(grid_search_model, model_file_name)
    
    
 

Saving Model:  Logistic_Regression.joblib
Saving Model:  SVM.joblib
Saving Model:  Decision_Tree_Classifier.joblib
Saving Model:  Random_Forest_Classifier.joblib
Saving Model:  KNN.joblib


In [317]:
# Loading model and running score analysis
models_scores_accuracy = []
models_scores_f1_score = []
models_scores_precision_score = []
models_scores_recall_score = []
models_scores_confusion_matrix = []

for model, model_name, param_grid in zip(models, model_names, param_grid_list):
        
    # loading the model and run for scores - score calculation
    model_file_name = model_name+".joblib"
    print("Loading Model: ", model_file_name)
    grid_search_model = joblib.load(model_file_name)

    # Making predictions using test data
    y_pred = grid_search_model.predict(X_test)
    
    print("Running test score >>>> working")
    # Calculate accuracy score for all models and storing in an array
    model_accuracy = accuracy_score(y_test, y_pred)
    models_scores_accuracy.append([model_name, model_accuracy])

    # Calculate f1 score for all models and storing in an array
    model_f1_score = f1_score(y_test, y_pred)
    models_scores_f1_score.append([model_name, model_f1_score])

    # Calculate precision score for all models and storing in an array
    model_precision_score = precision_score(y_test, y_pred)
    models_scores_precision_score.append([model_name, model_precision_score])

    # Calculate recall score for all models and storing in an array
    model_recall_score = recall_score(y_test, y_pred)
    models_scores_recall_score.append([model_name, model_recall_score])
 
    # Calculate confusion matrix for all models and storing in an array
    model_confusion_matrix = confusion_matrix(y_test, y_pred)
    models_scores_confusion_matrix.append([model_name, model_confusion_matrix])

    print("Working Successfull >>>>", model_name, "\n")


Loading Model:  Logistic_Regression.joblib
Running test score >>>> working
Working Successfull >>>> Logistic_Regression 

Loading Model:  SVM.joblib
Running test score >>>> working
Working Successfull >>>> SVM 

Loading Model:  Decision_Tree_Classifier.joblib
Running test score >>>> working
Working Successfull >>>> Decision_Tree_Classifier 

Loading Model:  Random_Forest_Classifier.joblib
Running test score >>>> working
Working Successfull >>>> Random_Forest_Classifier 

Loading Model:  KNN.joblib
Running test score >>>> working
Working Successfull >>>> KNN 



In [319]:
# Analyzing test scores
sorted_models_accuracy = sorted(models_scores_accuracy, key = lambda x: x[1], reverse=True )
print("\nsorted Model #accuracy scores:\n", sorted_models_accuracy)

sorted_models_f1_score = sorted(models_scores_f1_score, key = lambda x: x[1], reverse=True )
print("\nsorted Model #f1 scores:\n", sorted_models_f1_score)

sorted_models_precision_score = sorted(models_scores_precision_score, key = lambda x: x[1], reverse=True )
print("\nsorted Model #precision scores:\n", sorted_models_precision_score)

sorted_models_recall_score = sorted(models_scores_recall_score, key = lambda x: x[1], reverse=True )
print("\nsorted Model #recall scores:\n", sorted_models_recall_score)

for cm in models_scores_confusion_matrix:
    print("\n", cm[0], "\n",cm[1])


sorted Model #accuracy scores:
 [['Random_Forest_Classifier', 0.875], ['SVM', 0.8], ['Decision_Tree_Classifier', 0.8], ['KNN', 0.775], ['Logistic_Regression', 0.625]]

sorted Model #f1 scores:
 [['Random_Forest_Classifier', 0.5454545454545454], ['SVM', 0.5], ['Decision_Tree_Classifier', 0.5], ['KNN', 0.47058823529411764], ['Logistic_Regression', 0.44444444444444436]]

sorted Model #precision scores:
 [['Random_Forest_Classifier', 1.0], ['SVM', 0.5], ['Decision_Tree_Classifier', 0.5], ['KNN', 0.4444444444444444], ['Logistic_Regression', 0.3157894736842105]]

sorted Model #recall scores:
 [['Logistic_Regression', 0.75], ['SVM', 0.5], ['Decision_Tree_Classifier', 0.5], ['KNN', 0.5], ['Random_Forest_Classifier', 0.375]]

 Logistic_Regression 
 [[19 13]
 [ 2  6]]

 SVM 
 [[28  4]
 [ 4  4]]

 Decision_Tree_Classifier 
 [[28  4]
 [ 4  4]]

 Random_Forest_Classifier 
 [[32  0]
 [ 5  3]]

 KNN 
 [[27  5]
 [ 4  4]]
